In [1]:
#!pip install gensim==3.6.0

In [1]:
import gensim
gensim.__version__

c:\Users\R\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


'4.3.2'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir(r"G:\Ddrive\PG DBDA\12 Practical Machine Learning_\Datasets")

In [3]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import  Word2Vec

In [4]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t')

In [5]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
################### Replacing Apostrophe to Word #################
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
###################################################################

In [7]:
corpus = []
for i in range(0, dataset.shape[0]):
    review = dataset['Review'][i]
    review = decontracted(review)    
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
#    ps = PorterStemmer()
    review = review.split()
#    review = [word for word in review if not word in set(stops)]
#    review = ' '.join(review)
#    review = [review]
    corpus.append(review)

In [8]:
len(corpus)

1000

In [9]:
corpus[:3]

[['wow', 'loved', 'this', 'place'],
 ['crust', 'is', 'not', 'good'],
 ['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty']]

CBOW: sg = 0
Skip-Gram: sg = 1

In [10]:
model_r =  Word2Vec(corpus, min_count=1,sg=1)

In [16]:
corpus[0]

['wow', 'loved', 'this', 'place']

In [19]:
row_means = np.mean(model_r.wv[corpus[0]],axis=0)
row_means

array([-8.40492696e-02,  6.14943057e-02,  4.05316129e-02, -3.56965624e-02,
        5.85879572e-02, -2.07569838e-01,  4.63620126e-02,  2.75008410e-01,
       -8.40563625e-02, -1.17607683e-01, -1.98912174e-02, -2.27591276e-01,
       -3.71454731e-02,  1.73301045e-02,  1.23314913e-02, -4.97213602e-02,
        3.29901502e-02, -1.26803085e-01, -4.96878773e-02, -2.94826627e-01,
        3.59003134e-02,  2.90672970e-03,  1.43896520e-01, -1.28127038e-01,
       -2.55204830e-03,  6.48752153e-02, -1.11470848e-01, -2.92315688e-02,
       -1.44359201e-01,  9.02592763e-03,  1.71152800e-01, -4.00243420e-03,
        5.83990365e-02, -1.48053333e-01, -8.35967362e-02,  1.52561933e-01,
       -2.72447884e-04, -3.95296812e-02, -1.21733382e-01, -1.81834728e-01,
        5.49599975e-02, -7.66438544e-02, -2.94804387e-02, -9.80762392e-03,
        1.20021507e-01, -7.74494708e-02, -1.25559002e-01, -5.46433702e-02,
        2.62534013e-03,  1.60979033e-01,  6.18162826e-02, -1.43262997e-01,
       -9.72725637e-03, -

In [20]:
means = []
for i in corpus :
    row_means = np.mean(model_r.wv[i],axis=0)
    means.append(row_means)
means = np.array(means) 

X = means
y = dataset.iloc[:, 1]

In [21]:
X.shape

(1000, 100)

In [27]:
X[:,2:5]

array([[ 0.04053161, -0.03569656,  0.05858796],
       [ 0.04598686, -0.0301181 ,  0.05939537],
       [ 0.05270773, -0.03188234,  0.06066221],
       ...,
       [ 0.04916504, -0.04132613,  0.07506542],
       [ 0.04665598, -0.03999413,  0.06812572],
       [ 0.04741128, -0.03419282,  0.06046919]], dtype=float32)

In [28]:
y.shape

(1000,)

In [32]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, 
                                                    random_state=2021,
                                                    stratify=y)

from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=2021,oob_score=True,verbose=1)
model_rf.fit( X_train , y_train )

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.5s


RandomForestClassifier(oob_score=True, random_state=2021, verbose=1)

**Model Evaluation**


In [30]:
y_pred = model_rf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[93 57]
 [62 88]]
              precision    recall  f1-score   support

           0       0.60      0.62      0.61       150
           1       0.61      0.59      0.60       150

    accuracy                           0.60       300
   macro avg       0.60      0.60      0.60       300
weighted avg       0.60      0.60      0.60       300

0.6033333333333334
